In [1]:
#     IMPORT MODULES
## ------------------------------------------------------------------------

import dic
import numpy as np
import matplotlib.pyplot as plt
import math
import importlib
import sys  
import copy
import scipy as sp
import csv
import cv2 as cv
from scipy.linalg import solve
import os
from configparser import ConfigParser
import scipy.interpolate
import time
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import skimage as sk
from skimage import data
from skimage import transform as tf
from skimage.feature import (match_descriptors, corner_harris,
                             corner_peaks, ORB, plot_matches)
from skimage.color import rgb2gray
import statsmodels.api as sm
import patsy as ps
from patsy import dmatrices
from patsy import dmatrix
from numpy import loadtxt


In [2]:
#     SETUP SECTION
# ------------------------------------------------------------------------
# 
# Read/parse DIC setup parameters from settings file
configur = ConfigParser()
configur.read('Setup.ini')
sub_size = configur.getint('Subsets','SubsetSize')
sub_frequency = configur.getint('Subsets','SubsetFrequency')
GF_stddev = configur.getfloat('Filters','GaussianFilterStdDev')
GF_filtsize = configur.getint('Filters','GaussianFilterSize')
SF_order = configur.getint('Miscellaneous','ShapeFunctionOrder')
corr_refstrat = configur.getint('Miscellaneous','CorrelationReferenceStrategy')
calibrate = configur.getint('Miscellaneous','Calibration')
#store setup parameters
setup = np.array([])
setup = np.append(setup,sub_size)
setup = np.append(setup,SF_order)
setup = np.append(setup,GF_stddev)
setup = np.append(setup,GF_filtsize)
setup = np.append(setup,corr_refstrat)
setup = np.append(setup,sub_frequency)
#define images working directory
current_working_directory = os.getcwd()
folder = '\images'
image_location = current_working_directory + folder
##read and store images
print('---------------------------------------------------------------------------------------------------------------------------------')
N_images = 0
for filename in os.listdir(image_location):    
    #exec(f'img_{N_images} = cv.imread(filename,0)')
    #print('Image',N_images,':',filename)
    N_images = N_images + 1
F  =  cv.imread('star1_ref.tif',0)
F = cv.normalize(F.astype('double'), None, 0.0, 1.0, cv.NORM_MINMAX)
G = F[152:352,305:505]
F = F[150:350,300:500]
img_rows = F.shape[0]
img_columns = F.shape[1]
setup = np.append(setup,img_rows)
setup = np.append(setup,img_columns)
setup = np.append(setup,corr_refstrat)
setup = np.append(setup,calibrate)
print('Image dimensions:',img_rows,'X',img_columns,'pixels')
#display setup parameters
print('---------------------------------------------------------------------------------------------------------------------------------')
print ("Subset Size : ", sub_size)
print ("Subset Frequency : ", sub_frequency)
print ("Gaussian Filter Standard Deviation : ", GF_stddev)
print ("Gaussian Filter Size : ", GF_filtsize)
if SF_order == 1:
    print ("Shape Function/Warp Function: Affine transformation")
if corr_refstrat ==0:
    print("Correlation Reference Strategy : Absolute reference strategy (the first image is the reference for all correlation runs)")
elif corr_refstrat == 1:
    print("Correlation Reference Strategy : Incremental reference strategy (the preceding image is the reference image for current correlation run)")
if calibrate==0:
    print("Calibration : No")
elif calibrate == 1:
        print("Calibration : Yes")
print('Setup variabales: ',setup.shape[0],'\n',setup)
print('---------------------------------------------------------------------------------------------------------------------------------','\n')

---------------------------------------------------------------------------------------------------------------------------------
Image dimensions: 200 X 200 pixels
---------------------------------------------------------------------------------------------------------------------------------
Subset Size :  81
Subset Frequency :  5
Gaussian Filter Standard Deviation :  0.5
Gaussian Filter Size :  5
Shape Function/Warp Function: Affine transformation
Correlation Reference Strategy : Absolute reference strategy (the first image is the reference for all correlation runs)
Calibration : No
Setup variabales:  10 
 [ 81.    1.    0.5   5.    0.    5.  200.  200.    0.    0. ]
--------------------------------------------------------------------------------------------------------------------------------- 



In [4]:
# processed in MATLAB: import images, normalise images, compute image gradients
image_gradient_xy = np.array(np.gradient(F),dtype=float)
Fy = image_gradient_xy[0]
Fx = image_gradient_xy[1]
Fy = Fy/6
Fx = Fx/6

#
G = dic.DeformedImage(G,setup)
F = dic.ReferenceImage(F,setup)
F.F_grady = Fy
F.F_gradx = Fx
G.sub_centres[0,:] = F.sub_centres[0,:] + np.round(F.P[0,:])
G.sub_centres[1,:] = F.sub_centres[1,:] + np.round(F.P[3,:])
G.P[0,:], G.P[3,:] = dic.EstimateDisplacementsFourier(F,G)
print(F.x)
print(F.y)






IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
print(G.P[0,:])

[-5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5.
 -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5.
 -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5.
 -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5.
 -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5.
 -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5.
 -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5.
 -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5.
 -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5.
 -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5.
 -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5.
 -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5.
 -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5.
 -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5. -5

In [ ]:
print(G.P[3,:])

[-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.
 -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.
 -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.
 -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.
 -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.
 -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.
 -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.
 -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.
 -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.
 -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.
 -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.
 -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.
 -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.
 -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2

In [ ]:
## G.P[0,0], G.P[3,0], coeff = EstimateDisplacements(F, G, 0, sub_size)
#print(coeff)
print('u0, v0: Initial estimate of displacments:')
print(G.P[0,0], G.P[3,0])
print('G.P, Initial shape function parameters')
print(G.P[:,0])
#perform correlation on first subset
N_subsets = F.sub_centres.shape[1]
for i in range(0,N_subsets):
#extract subset from F
    f, f_mean, f_tilde, dfdx, dfdy  = dic.RefSubsetInfo(F,i)
    Hess, fgrad_X_dWdP = dic.Hessian(dfdx, dfdy, F)
    deltaP =  np.ones([6,1])
    itera = 0
    while itera < 100:
        #deform square subset with linear transformation based on current estimation of warp function parameters (u   ux  uy  v   vx  vy)
        deformed_subset = dic.AffineTrans(G,i)
        #extract subset from interpolated G
        g, g_mean, g_tilde = dic.DefSubsetInfo(G, deformed_subset, i)
        stop_val = dic.StopCriteria(deltaP, 0.5*(G.sub_size-1))
        if stop_val < 1e-4:
            #Zero-Mean Normalised Cross Correlation Criteria
            G.corr_coeff[0,i] = 1 - sum(((f[:]-f_mean)/f_tilde-(g[:]-g_mean)/g_tilde)**2)/2
            G.stop_val[0,i] = stop_val
            G.iterations[0,i]  = itera
            break
        else:
            Jacobian = np.dot(fgrad_X_dWdP.T, (f[:]-f_mean-f_tilde/g_tilde*(g[:]-g_mean)))
            deltaP = np.linalg.solve(-Hess, Jacobian) #-1*np.linalg.inv(Hess)*Jacobian.T
            Pupdate = dic.UpdateSFP(G.P[:,i], deltaP)
            G.P[:,i:i+1] = Pupdate
        itera = itera + 1
    
print(itera, ' iteration complete')
print(i+1, ' subsets out of ',N_subsets,' correlated')
print('u, v: New G.P')
print(G.P[:,0])

u0, v0: Initial estimate of displacments:
-5.0 -2.0
G.P, Initial shape function parameters
[-5.  0.  0. -2.  0.  0.]


KeyboardInterrupt: 

In [ ]:
print('realdisplacements: u0 = 5, v0 = -2\n')
print(i+1,'th subset ',itera,' iterations')
print(G.iterations)
print(G.P[:,i])



realdisplacements: u0 = 5, v0 = -2

150 th subset  1  iterations
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

In [ ]:
N_points = deformed_subset.shape[0]
centerx, centery = np.array([G.sub_centres[0][i],G.sub_centres[1][i]])
print('Subset centres, row, column:', centery, centerx)
y = centery*np.ones(N_points) + deformed_subset[:,1]
x = centerx*np.ones(N_points) + deformed_subset[:,0]

print(y)




Subset centres, row, column: 45 45
[ 1.64805281  2.74883048  3.84960816 ... 90.41823038 91.51900806
 92.61978573]


In [ ]:
print(x)

[-2.35714866 -2.31586431 -2.27457996 ... 98.17892232 98.22020667
 98.26149101]
